In [ ]:
def ComputeCFactor(parcel_list, grid, ggg, cp, year, output_interval="M",
                   ffull_output=False, output_map="Results", multiprocessing=False, fid="perceel_id"):
    """
    Script to computes C factor per parcel, based on formula's Verbist, K. (2004). Computermodel RUSLE C-factor.

    Parameters
    ----------
        'parcel_list' (pd df): list of parcels, with coupled crops and crop properties
            'REF_ID' (int): id of parcel according to 'perceelsregistratie'
            'perceel_id' (int): generic added id
            'jaar' (int): years considered
            'type' (string): type of crop, voorteelt, hoofdteelt, nateelt
            'GWSCOD' (int): code of the prop, see file 'teelt_eigenschappen.csv'
            'groep_id' (int/float): id of crop group
        'grid' (pd df): computation grid with columns
            'bdate' (datetime): begindate node
            'edate' (datetime): enddate node
            'bmonth' (datetime): month node begin
            'bday' (datetime): day node begin
            'rain' (float): summed rainfall node (mm)
            'temp' (float): average temperature node (degree Celsius)
            'Rhm' (float): erosion index ((MJ mm)/(ha h) as defined in see Verstraeten, G., Poesen, J., Demarée, G.,
             & Salles, C. (2006). Long-term (105 years) variability " \
             In rain erosivity as derived from 10-min rainfall depth data for Ukkel (Brussels, Belgium):
             Implications for assessing soil erosion rates. Journal of Geophysical Research Atmospheres, 111(22), 1–11.)
        'year' (int): year for which simulation is run
        'ggg' (pd df):
            'ggg_id' (int): id coupling crop growth properties
            'dagen_na' (int): number of days after sowing
            'Fc' (float): bodembedekking (m2/m2)
            'H' (float): effectieve valhoogte (m)
        'gts' (pd df):  teelt zaai en oogstdata + data groeicurves
            'groep_id' (int/float): id of crop group
            'zaaidatum1' (int): 'average' sowing date of crop or begin range sowing date
            'zaaidatum2' (int): end range sowing data
            'dagen_tot_oogst' (int): average number of days untill harvest
            'ggg_id' (int): id of crop growth curve (gewasgroeicurve)
        'cp' (pd df): crop properties (Bsi, alpha, p, Ri, see definitions
            1. Verbist, K. (2004). Computermodel RUSLE C-factor.
            2. Verstraeten, G., Van Oost, K., Van Rompaey, A., Poesen, J., & Govers, G. (2001). Integraal land-
                en waterbeheer in landelijke gebieden met het oog op het beperken van bodemverlies en modderoverlast
                ( proefproject gemeente Gingelom ) Integraal land- en waterbeheer in landelijke gebieden met het oog
                op het beperken van bodemverlies en m. Laboratory for Experimental Geomorphology (2014), 67.
            'groep_id' (int/float): id of crop group
            'groep_name' (string): name of crop group
        'output_interval' (str): output interval for C-factor (!= interval computation grid), either monthly 'M',
                                semi-monthly'SMS' or yearly 'Y'
        'full_ouput' (bool): write grid to disk for every parcel (default False)
        'output_map' (string): map to write outputs to
        'multiprocessing' (boolean):  use multiple cores (True= yes)
        "perceel_id" (str): flag for perceels_id

    Returns
    -------
         XXXXX

    """

    # (SG) Only compute parcels for which groep_id is known, and for which a main crop is defined for year i
    parcel_list_comp = parcel_list[parcel_list["compute"] == 1]

    # compute C
    if not multiprocessing:
        out = single(parcel_list_comp, grid, ggg,  cp, year, output_interval=output_interval,
                     ffull_output=ffull_output, output_map=output_map, fid=fid)
    else:
        out = multi(parcel_list_comp, grid, ggg,  cp, year, output_interval=output_interval,
                    ffull_output=ffull_output, output_map=output_map, fid=fid)

    return out


def multi(parcel_list, grid, ggg, cp, year, output_interval="M",
          ffull_output=False, output_map="Results", fid="perceel_id"):

    import multiprocessing

    ncpu = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(ncpu)
    un_parcels = parcel_list[fid].unique()
    length_job = int(np.ceil(len(un_parcels)/ncpu))
    ind = [np.min([len(un_parcels), i*length_job]) for i in np.arange(ncpu+1)]
    jobs = [0.] * ncpu
    output = []

    for i in range(len(jobs)):
        parcels_i = un_parcels[ind[i]:ind[i+1]]
        jobs[i] = pool.apply_async(eval('single'),
                                   (parcel_list.loc[parcel_list[fid].isin(parcels_i)],
                                    grid, ggg, cp, year, output_interval, ffull_output, output_map, fid, i))
    pool.close()

    for i in range(len(jobs)):
        temp = jobs[i].get()
        output.append(temp)

    return pd.concat(output)


def single(parcel_list, grid, ggg, cp, year, output_interval="M", ffull_output=False,
           output_map="Results", fid="perceel_id", processor=1):

    # note: calculations are computed on a forward grid,
    # e.g. the record with timetag 01/01 are calculations for coming X days
    # initialize columns
    SLRcol = ["SLR_%i" % i for i in range(24)]
    Hcol = ["H_%i" % i for i in range(24)]
    Fccol = ["Fc_%i" % i for i in range(24)]
    Rcol = ["R_%i" % i for i in range(24)]

    # (SG) filter duplicates
    # parcel_list = parcel_list[["GWSCOD","GWSNAM","NR","type","jaar","perceel_id","groep_id","voorteelt",
    #                          "hoofdteelt","nateelt"  groente  groenbedekker  meerjarige_teeltonvolledig
    #                          teeltgegevens_beschikbaar  compute]].drop_duplicats(columns=[[]])

    # ids = parcel_list[parcel_list["perceel_id"]==14432].index[0]
    # parcel_list = parcel_list.loc[ids:]

    un_par = parcel_list[fid].unique()
    out = []

    ind = 0
    # condn = 5
    # starttime = time.time()

    for parcel_id in un_par:
        print("[ComputeC] Running parcel_id %i on processor %i (%.2f %% complete)"
              % (parcel_id, processor, (ind/len(un_par)*100)))
        cond = parcel_list[fid] == parcel_id
        C, SLR, H, Fc, R = compute_C(parcel_list[cond], grid, ggg, cp, year, parcel_id,
                                     output_interval=output_interval, ffull_output=ffull_output, output_map=output_map)

        # (SG) print parcel to disk
        if ffull_output:
            parcel_list[cond].to_csv(os.path.join(output_map, "%i_rotation_scheme.csv" % int(parcel_id)))

        out.append(np.array([parcel_id]+[C]+SLR.tolist()+Fc.tolist()+H.tolist()))
        ind += 1

    out = pd.DataFrame(np.stack(out), columns=[fid, "C"] + SLRcol + Fccol + Hcol)
    # ind += 1

    # cond = np.round(100 * ind / len(parcel_list))

    # if cond >= condn:
    #    print("%i runs of %i done (%i %%), time elapsed (min): %.2f" % (
    #    ind, len(un_par), condn, (time.time() - starttime) / 60))
    #    condn += 5
    return out

In [ ]:
from cfactor.io import prepare_grid
from cfactor.util import create_dir
from cfactor.cfactor import compute_Ru, compute_SR, compute_CC,compute_SM, compute_PLU, compute_SLR, weight_SLR

In [ ]:
def compute_C(
    parcel,
    grid,
    ggg,
    cp,
    year,
    parcel_id,
    output_interval="M",
    output_map="Results",
    ffull_output=False,
):
    """
    Computes C factor based on formula's Verbist, K. (2004). Computermodel RUSLE C-factor.

    Parameters
    ----------
        'parcel' (pd df): considered parcels, see parameter ``parcel_list`` in :func:`ComputeCFactor`.
        'grid' (pd df): see parameter ``grid`` in :func:`ComputeCFactor`.
        'ggg' (pd df): see parameter ``ggg`` in :func:`ComputeCFactor`.
        'gts' (pd df): see parameters 'gts in :func:`ComputeCFactor`
        'cp' (pd df): see parameters 'cp' in :func:`ComputeCFactor`
        'year' (int): see parameters 'year' in :func:`ComputeCFactor`
        'parcel_id' (int): id of parcel
        'output_interval' (str,opt): see parameter ``output_interval`` in :func:`ComputeCFactor`.
        'output_map' (str,opt): see parameters 'output_map' in :func:`ComputeCFactor`
        'fful_output (bool,opt): see parameters 'fful_outpit' in :func:`ComputeCFactor`

    Returns
    -------
         'C' (pd df): see factor, aggregrated according to output_interval
    """

    create_dir("", [output_map])

    # (SG) prepare grid
    grid = prepare_grid(
        parcel, grid, ggg, cp, year, parcel_id, output_map, ffull_output=ffull_output
    )

    grid["f1_N"], grid["f2_EI"], grid["Ru"] = compute_Ru(
        grid.loc[:, "Ri_tag"].values.flatten(),
        grid.loc[:, "Ri"],
        grid.loc[:, "rain"],
        grid.loc[:, "Rhm"],
    )

    grid["SR"] = compute_SR(grid.loc[:, "Ru"])

    grid["a"], grid["Bsb"], grid["Sp"], grid["W"], grid["F"], grid["SC"] = compute_SC(
        grid.loc[:, "har_tag"],
        grid.loc[:, "bdate"],
        grid.loc[:, "edate"],
        grid.loc[:, "rain"],
        grid.loc[:, "temp"],
        grid.loc[:, "p"],
        grid.loc[:, "Bsi"],
        grid.loc[:, "alpha"],
        grid.loc[:, "Ru"],
    )

    grid["CC"] = compute_CC(grid.loc[:, "H"], grid.loc[:, "Fc"])

    grid["SM"] = compute_SM(grid.loc[:, "SM"])

    grid["PLU"] = compute_PLU(grid.loc[:, "PLU"])

    grid["SLR"] = compute_SLR(
        grid.loc[:, "SC"],
        grid.loc[:, "CC"],
        grid.loc[:, "SR"],
        grid.loc[:, "SM"],
        grid.loc[:, "PLU"],
    )

    if ffull_output:
        grid.to_csv(os.path.join(output_map, "%i_calculations.csv" % parcel_id))

    # (SG) only consider year i
    grid = grid.loc[grid["bdate"].dt.year == year]

    C = weight_SLR(grid["SLR"], grid["Rhm"], grid["bdate"], output_interval)

    return (
        C,
        grid["SLR"].values.flatten(),
        grid["H"].values.flatten(),
        grid["Fc"].values.flatten(),
        grid["Rhm"].values.flatten(),
    )